In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Load the dataset
data_dir = 'E:/PPInput'
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the models to evaluate
models_to_evaluate = {
    'SimpleCNN': nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(16, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Flatten(),
        nn.Linear(64 * 28 * 28, 128),
        nn.ReLU(),
        nn.Linear(128, 3)
    ),
    'EfficientNetB0': models.efficientnet_b0(pretrained=True),
    'EfficientNetB1': models.efficientnet_b1(pretrained=True),
    'DenseNet121': models.densenet121(pretrained=True),
    'InceptionV3': models.inception_v3(pretrained=True)
}

# Modify the final layer of each pre-trained model
for model_name, model in models_to_evaluate.items():
    if model_name == 'SimpleCNN':
        continue
    if model_name in ['EfficientNetB0', 'EfficientNetB1']:
        num_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_features, 3)
    elif model_name == 'DenseNet121':
        num_features = model.classifier.in_features
        model.classifier = nn.Linear(num_features, 3)
    elif model_name == 'InceptionV3':
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 3)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Training and evaluation function
def train_and_evaluate_model(model, train_loader, val_loader, num_epochs=5):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Evaluate the model
    model.eval()
    val_labels = []
    val_preds = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(val_labels, val_preds)
    precision = precision_score(val_labels, val_preds, average='weighted')
    recall = recall_score(val_labels, val_preds, average='weighted')
    f1 = f1_score(val_labels, val_preds, average='weighted')

    return accuracy, precision, recall, f1

# Iterate over the models and print metrics
for model_name, model in models_to_evaluate.items():
    print(f"Evaluating {model_name}...")
    accuracy, precision, recall, f1 = train_and_evaluate_model(model, train_loader, val_loader)
    print(f"Model: {model_name}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


Evaluating SimpleCNN...
Model: SimpleCNN, Accuracy: 0.9730, Precision: 0.9749, Recall: 0.9730, F1 Score: 0.9729
Evaluating EfficientNetB0...
Model: EfficientNetB0, Accuracy: 0.9730, Precision: 0.9749, Recall: 0.9730, F1 Score: 0.9729
Evaluating EfficientNetB1...
Model: EfficientNetB1, Accuracy: 0.8311, Precision: 0.8534, Recall: 0.8311, F1 Score: 0.8266
Evaluating DenseNet121...
Model: DenseNet121, Accuracy: 0.9865, Precision: 0.9870, Recall: 0.9865, F1 Score: 0.9865
Evaluating InceptionV3...


RuntimeError: Calculated padded input size per channel: (3 x 3). Kernel size: (5 x 5). Kernel size can't be greater than actual input size